I'll first import the mnist_load notebook just like a python module.
This way I don't have to worry about extracting and reshaping
the train and test data again.

In [1]:
import numpy as np
import sys

In [2]:
#It has the code to import notebooks;
import ipynb_reader
#my notebook that reads and reshape the data;
import mnist_load as load

train_images_array = load.train_images_array;
train_labels_array = load.train_labels_array;
test_images_array = load.test_images_array;
test_labels_array = load.test_labels_array;

importing Jupyter notebook from mnist_load.ipynb
(0, 0, 8, 3)
images_n:60000
Rows: 28Columns: 28
Images: (1000, 28, 28)
Labels: (1000,)
(1000, 10)
(0, 0, 8, 3)
Rows: 28Columns: 28
Images: (1000, 28, 28)
Labels: (1000,)
(1000, 10)
(4,)
(1000, 784)
(1000, 784)


In [3]:
#change numpy print threshold;
#set to 'inf' if you wish to print the whole vector/matrix; 
#np.set_printoptions(threshold='1000');

Now I'm going to effectively start the challenge:
Code our previous neural network logic using the two new methods seen in this chapter i.e. Dropout and mini-batched gradient descent, from of memory.

In [40]:
np.random.seed(1);

relu = lambda x : (x > 0) * x;
relu2deriv = lambda x : (x > 0);

batch_size = 100;
alpha, hidden_size, iterations = (.01, 387, 500);
pixels_per_image, labels_per_image = (784, 10);

synapse_0 =  .2 * np.random.random((pixels_per_image, hidden_size)) - .1;
synapse_1 =  .2 * np.random.random((hidden_size, labels_per_image)) - .1;

for iteration in range(iterations):
    train_msquared_error, train_correct_cnt = .0, .0;
    for index in range(int(len(train_images_array)/batch_size)):
        batch_start, batch_end = index * batch_size, \
                                    (index + 1) * batch_size;
        #forward propagation
        layer_0 = train_images_array[batch_start:batch_end];
        layer_1 = relu(layer_0.dot(synapse_0));
        #dropout mask
        dropout_mask = np.random.randint(2, size=layer_1.shape);
        layer_1 *= dropout_mask * 2;
        
        layer_2 = layer_1.dot(synapse_1);
        
        train_msquared_error += np.sum((layer_2 - \
                                train_labels_array[batch_start:batch_end]) ** 2);
        for key in range(batch_size):
            train_correct_cnt += np.argmax(layer_2[key:key+1]) \
                                    == np.argmax( \
                                     train_labels_array[batch_start+key:batch_start+key+1]);
        
        #Global Correlation Summarization
        #Backpropagation or cross-communication
        layer_2_delta = (layer_2 - train_labels_array[batch_start:batch_end]) / batch_size;
        layer_1_delta = layer_2_delta.dot(synapse_1.T) * relu2deriv(layer_1);
        
        ##is not necessary giving that layer_1 alredy store result within, after dropout multiplication.
        #layer_1_delta *= dropout_mask * 2;        

        #Local Correlation Summarization
        ##Gradient Descent optimization
        ###Gradient
        synapse_1_delta = layer_1.T.dot(layer_2_delta);
        synapse_0_delta = layer_0.T.dot(layer_1_delta);
        ##Descent
        synapse_1 -= synapse_1_delta * alpha;
        synapse_0 -= synapse_0_delta * alpha;
    
    #time for inference
    if(iteration % 10 == 9):
        test_msquared_error, test_correct_cnt = 0, 0;
        for index in range(len(test_images_array)):
            layer_0 = test_images_array[index:index+1];
            layer_1 = relu(layer_0.dot(synapse_0));
            layer_2 = layer_1.dot(synapse_1);
            
            test_msquared_error += np.sum((layer_2 
                                            - test_labels_array[index:index+1]) ** 2);
            test_correct_cnt += np.argmax(layer_2) \
                                        == np.argmax(test_labels_array[index:index+1]);
        sys.stdout.write('\r' + \
                             'I: ' + str(iteration) + \
                             ' Test_Error: ' + str(test_msquared_error/len(test_images_array))[0:5] + \
                             ' Test_Accuracy: ' + str(test_correct_cnt/len(test_images_array)) + \
                             ' Train_Error: ' + str(train_msquared_error/len(train_images_array))[0:5] + \
                             ' Train_Accuracy: ' + str(train_correct_cnt/len(train_images_array)));
                             

I: 499 Test_Error: 6.635 Test_Accuracy: 0.739 Train_Error: 3.384 Train_Accuracy: 0.816